In [1]:
import pandas as pd
import datetime as dt

In [2]:
f = lambda s: dt.datetime.strptime(s,'%d-%m-%Y %H:%M:%S')
# df_bn5 = pd.read_csv("data/historical/BankNiftyFut-Sep-5min-20200908.csv", sep="\t", parse_dates=['Date Time'], date_parser=f, encoding="utf_16_LE") 
df_bn5 = pd.read_csv("data/historical/NiftyFut-Sep-5min-20200910.csv", sep="\t", parse_dates=['Date Time'], encoding="utf_16_LE") 

#### Remove last empty column and set date time as index

In [3]:
df_bn5 = df_bn5[df_bn5.columns[:-1]]
df_bn5.set_index('Date Time', inplace=True)

## Input the evaluation date below

In [4]:
eval_date = "2020-09-10"

#### Extract a separate dataframe for the evaluation date

In [5]:
today_df_bn5 = df_bn5[eval_date]

#### Apply rolling mean columns for data

In [6]:
today_df_bn5['SMA6'] = today_df_bn5['Close'].rolling(6).mean()
today_df_bn5['SMA21'] = today_df_bn5['Close'].rolling(21).mean()

<ipython-input-6-f2f66b587a2f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  today_df_bn5['SMA6'] = today_df_bn5['Close'].rolling(6).mean()
<ipython-input-6-f2f66b587a2f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  today_df_bn5['SMA21'] = today_df_bn5['Close'].rolling(21).mean()


In [7]:
today_df_bn5.head(5)

,Open,Low,High,Close,Volume,SMA6,SMA21
Date Time,,,,,,,
2020-09-10 09:15:00,11341.15,11331.35,11359.95,11350.10,596028,NaN,NaN
2020-09-10 09:20:00,11350.85,11348.65,11358.55,11354.00,307350,NaN,NaN
2020-09-10 09:25:00,11353.95,11348.35,11358.60,11350.50,202050,NaN,NaN
2020-09-10 09:30:00,11350.95,11346.00,11356.70,11354.95,141075,NaN,NaN
2020-09-10 09:35:00,11354.50,11352.55,11366.10,11360.95,274800,NaN,NaN


### Strategy function

In [8]:
def backtest_Intraday_5min_strategy(df):
    SELL = False
    BUY = False
    SELL_PRICE = 0
    BUY_PRICE = 0
    GAIN_POINTS = 0
    SELL_STOPLOSS = 0
    BUY_STOPLOSS = 0
    STOPLOSS = 25
    SMA6_dead_band = 10
    Total_Signals = 0
    Total_Sell_Signal = 0
    Total_Buy_Signal = 0
    start_time = dt.datetime.strptime("09:20", "%H:%M").time()
    close_time = dt.datetime.strptime("15:15", "%H:%M").time()
    entry_close_time = dt.datetime.strptime("15:00", "%H:%M").time()
    print("Time\t\t\t", "Close\t\t", "SMA6\t\t", "SMA21\t\t", "SIGNAL\t", "TYPE\t\t",  "Points\t",)
    
    for row in df.itertuples():
        SMA6_upper_band = row.SMA6 + SMA6_dead_band
        SMA6_lower_band = row.SMA6 - SMA6_dead_band
        
        # SELL STOPLOSS
        if row.Index.time() >= start_time and row.Index.time() <= close_time: 
            points = 0
            if SELL and row.Close > SELL_STOPLOSS:
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Buy_Signal += 1
                points = SELL_PRICE - row.Close
                GAIN_POINTS = GAIN_POINTS + points
                SELL_PRICE = 0
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "SELL STOPLOSS\t", round(points,2))
            
            # BUY STOPLOSS
            points = 0
            if BUY and row.Close < BUY_STOPLOSS:
                points = row.Close - BUY_PRICE
                GAIN_POINTS = GAIN_POINTS + points
                BUY_PRICE = 0
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Sell_Signal += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "BUY STOPLOSS\t", round(points,2))
            
            # SELL ENTRY
            points = 0
            if row.Close < SMA6_lower_band and \
                    row.Index.time() < entry_close_time and \
                    SELL is False:
                SELL_PRICE = row.Close
                SELL_STOPLOSS = SELL_PRICE + STOPLOSS
                if BUY_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                SELL = True
                BUY = False
                Total_Sell_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL ENTRY\t", round(points,2))
             
            # BUY ENTRY
            points = 0
            if row.Close > SMA6_upper_band and \
                    row.Index.time() < entry_close_time and \
                    BUY is False:
                BUY_PRICE = row.Close
                BUY_STOPLOSS = BUY_PRICE - STOPLOSS
                if SELL_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                BUY = True
                SELL = False
                Total_Buy_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t", row.Close,  "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY ENTRY\t", round(points,2))
            
            # DAY Close
            points = 0    
            if row.Index.time() >= close_time:
                if SELL:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = SELL_PRICE - row.Close
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY DAYCLOSE\t", round(points,2))

                if BUY:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = row.Close - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL DAYCLOSE\t", round(points,2))
                    
                    
    
    print("\n\nTotal Signals:\t\t", Total_Signals)
    print("Total Sell Signals:\t", Total_Buy_Signal)
    print("Total Buy Signals:\t", Total_Sell_Signal)
    print("Total Points Gained:\t", round(GAIN_POINTS,2))

In [9]:
backtest_Intraday_5min_strategy(today_df_bn5)

Time			 Close		 SMA6		 SMA21		 SIGNAL	 TYPE		 Points	
2020-09-10 10:25:00 	 11370.8 	 11360.25 	 nan 	 BUY	 BUY ENTRY	 0
2020-09-10 12:00:00 	 11413.65 	 11423.99 	 11404.75 	 SELL	 SELL ENTRY	 42.85
2020-09-10 12:40:00 	 11439.4 	 11421.53 	 11418.17 	 BUY	 SELL STOPLOSS	 -25.75
2020-09-10 12:40:00 	 11439.4 	 11421.53 	 11418.17 	 BUY	 BUY ENTRY	 0
2020-09-10 12:55:00 	 11405.0 	 11421.12 	 11419.72 	 SELL	 BUY STOPLOSS	 -34.4
2020-09-10 12:55:00 	 11405.0 	 11421.12 	 11419.72 	 SELL	 SELL ENTRY	 0
2020-09-10 14:05:00 	 11396.15 	 11384.3 	 11399.41 	 BUY	 BUY ENTRY	 8.85
2020-09-10 15:15:00 	 11457.2 	 11446.15 	 11415.88 	 SELL	 SELL DAYCLOSE	 61.05


Total Signals:		 8
Total Sell Signals:	 5
Total Buy Signals:	 3
Total Points Gained:	 52.6
